# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [96]:
# your code here
Odf = pd.read_csv("../Orders.csv")
Odf.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [97]:
# your code here
#How to aggregate the amount_spent for unique customers
customers_agg = Odf.groupby("CustomerID").sum()
customers_agg.head()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40


In [98]:
#How to select customers whose aggregated amount_spent is in a given quantile range?
customers_agg = customers_agg[["amount_spent"]]
customers_agg.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [99]:
#Sum of amount spent 
customers_agg[["amount_spent"]].sum()

amount_spent    8911407.904
dtype: float64

In [100]:
quantiles_custom = customers_agg.quantile([1,.90,.75,.50,.25,0]) #Given quantil range 90 percent 75 percent 50 percent ...
quantiles_custom

,amount_spent
1.00,280206.020
0.90,3646.164
0.75,1661.640
0.50,674.450
0.25,307.245
0.00,0.000


In [101]:
qcustom_090 = int(customers_agg.quantile(.90)["amount_spent"])
qcustom_075 = int(customers_agg.quantile(.75)["amount_spent"])
print(qcustom_090,qcustom_075)

3646 1661


In [102]:
#The 90% quantile is achieved when the total amount spent by customer is greater than 3646€,
#and the 75% quantile is achieved when a greater spent than 1661€ is done by customer.
#Hence, the 0,90% will be assumed for VIP customers and the 75%-90 are preferred.50-75% Normal
def find_cust(amount):
    if amount >= qcustom_090:
        return "VIP"
    elif qcustom_075 <= amount < qcustom_090:
        return "Preferred"
    return "Normal"

In [103]:
customers_agg["Customer Type"] = customers_agg["amount_spent"].apply(find_cust)
customers_agg

,amount_spent,Customer Type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,VIP
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Normal
...,...,...
18280,180.60,Normal
18281,80.82,Normal
18282,178.05,Normal


In [73]:
Preferred_customers = customers_agg[(customers_agg['amount_spent']>= customers_quantile.iloc[1,0]) & 
                                    (customers_agg['amount_spent']< customers_quantile.iloc[0,0]) ]
Preferred_customers

,amount_spent,Customer Type
CustomerID,,
12348,1797.24,Preferred
12349,1757.55,Preferred
12352,2506.04,Preferred
12356,2811.43,Preferred
12360,2662.06,Preferred
...,...,...
18259,2338.60,Preferred
18260,2643.20,Preferred
18272,3078.58,Preferred


In [67]:
customers_quantile = customers_agg.quantile([.90, .75])
customers_quantile

,amount_spent
0.90,3646.164
0.75,1661.640


In [71]:
VIP_customers = customers_agg[customers_agg['amount_spent']>= customers_quantile.iloc[0,0]]
VIP_customers

,amount_spent,Customer Type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,VIP
12357,6207.67,VIP
12359,6372.58,VIP
12362,5226.23,VIP
...,...,...
18223,6484.54,VIP
18225,5509.12,VIP
18226,5228.05,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [74]:
VIP_list = list(VIP_customers.index)
pref_list = list(Preferred_customers.index)

In [83]:
# your code here
VIP_cust_country = Odf[Odf['CustomerID'].isin(VIP_list)]
VIP_cust_country.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [51]:
VIP_list = list(VIP_customers.index)
VIP_cust_country = Odf[Odf['CustomerID'].isin(VIP_list)]
VIP_cust_country

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397904,541889,581585,22466,2011,12,5,12,fairy tale cottage night light,12,2011-12-09 12:31:00,1.95,15804,United Kingdom,23.40
397905,541890,581586,22061,2011,12,5,12,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,23.60
397906,541891,581586,23275,2011,12,5,12,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,United Kingdom,30.00
397907,541892,581586,21217,2011,12,5,12,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,United Kingdom,214.80


In [88]:

# your code here
df_countries = Odf[["Country","CustomerID"]]#,"CustomerID",
df_countries.head()


,Country,CustomerID
0,United Kingdom,17850
1,United Kingdom,17850
2,United Kingdom,17850
3,United Kingdom,17850
4,United Kingdom,17850


In [90]:
customers_agg = df_countries.drop_duplicates()
customers_agg_countries = customers_agg.sort_values("CustomerID")
customers_agg_countries.head(10)

,Country,CustomerID
37126,United Kingdom,12346
10516,Iceland,12347
23010,Finland,12348
358156,Italy,12349
49326,Norway,12350
57551,Norway,12352
136162,Bahrain,12353
113099,Spain,12354
124376,Bahrain,12355
37090,Portugal,12356


In [104]:
merged_countries_VIP = customers_agg_countries.merge(customers_agg,left_on="CustomerID",right_on="CustomerID")
merged_countries_VIP.head(20)

,Country,CustomerID,amount_spent,Customer Type
0,United Kingdom,12346,77183.60,VIP
1,Iceland,12347,4310.00,VIP
2,Finland,12348,1797.24,Preferred
3,Italy,12349,1757.55,Preferred
4,Norway,12350,334.40,Normal
5,Norway,12352,2506.04,Preferred
6,Bahrain,12353,89.00,Normal
7,Spain,12354,1079.40,Normal
8,Bahrain,12355,459.40,Normal
9,Portugal,12356,2811.43,Preferred


In [106]:
merged_countries_VIP = merged_countries_VIP.loc[(merged_countries_VIP["Customer Type"]=="VIP")]
merged_countries_VIP["Country"].unique()

array(['United Kingdom', 'Iceland', 'Switzerland', 'Cyprus', 'Belgium',
       'Australia', 'Spain', 'Finland', 'Denmark', 'Austria', 'Norway',
       'France', 'Germany', 'Israel', 'Singapore', 'Japan', 'Portugal',
       'Poland', 'EIRE', 'Netherlands', 'Channel Islands', 'Sweden'],
      dtype=object)

In [109]:
len(set(merged_countries_VIP["Country"]))

22

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here